# Chat Analysis Dashboard

This notebook provides comprehensive analysis and visualization of your processed AI chat conversations.

## Setup and Data Loading

In [7]:
import json
import sqlite3
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set up plotting styles
plt.style.use('default')
sns.set_palette("husl")

print("📊 Analysis libraries loaded successfully!")

📊 Analysis libraries loaded successfully!


In [8]:
# Environment and Kernel check (recommended)
import sys
from pathlib import Path as _Path

print("\n🧪 Environment check:")
print(f" • Python executable: {sys.executable}")
_in_venv = (getattr(sys, 'base_prefix', sys.prefix) != sys.prefix) or hasattr(sys, 'real_prefix')
print(f" • In virtualenv: {_in_venv}")
# Heuristic: suggest using the repo venv if not active
_repo_venv = _Path('chat_analysis_env')
if _repo_venv.exists():
    expected_name = 'chat_analysis_env'
    is_expected = expected_name in sys.executable or expected_name in sys.prefix
    if not is_expected:
        print(" ⚠️ Notebook kernel does not appear to be using the 'chat_analysis_env' virtual environment.")
        print("    To use the correct environment in Jupyter:")
        print("      1) source chat_analysis_env/bin/activate  (Windows: chat_analysis_env\\Scripts\\activate)")
        print("      2) python -m ipykernel install --user --name chat_analysis_env --display-name \"Python (chat_analysis_env)\"")
        print("      3) In Jupyter, select Kernel → Change kernel → Python (chat_analysis_env)")

# Quick dependency/resource checks
_missing = []
try:
    import spacy  # type: ignore
    try:
        import en_core_web_sm  # type: ignore
        _has_spacy_model = True
    except Exception:
        # Try loading by name in case it's already linked
        try:
            spacy.load("en_core_web_sm")
            _has_spacy_model = True
        except Exception:
            _has_spacy_model = False
    if not _has_spacy_model:
        _missing.append("spaCy model 'en_core_web_sm'")
except Exception:
    _missing.append("spaCy package")

try:
    import nltk  # type: ignore
    try:
        nltk.data.find('sentiment/vader_lexicon')
        _has_vader = True
    except LookupError:
        _has_vader = False
    if not _has_vader:
        _missing.append("NLTK VADER lexicon")
except Exception:
    _missing.append("NLTK package")

# Sentence transformers optional for the notebook, required for embeddings
try:
    import sentence_transformers  # type: ignore
except Exception:
    print(" ℹ️ Optional: 'sentence-transformers' not importable in this kernel. Embedding visualizations may be limited if used here.")

if _missing:
    print(" ⚠️ Missing components detected:")
    for m in _missing:
        print(f"   - {m}")
    print(" 👉 Install inside the active environment:")
    print("    • pip install -r requirements.txt")
    print("    • python -m spacy download en_core_web_sm")
    print("    • python -c \"import nltk; nltk.download('vader_lexicon')\"")

# Locate and validate database
from pathlib import Path

db_path = Path('chat_analysis.db')
if not db_path.exists():
    print("⚠️ chat_analysis.db not found in the project root.")
    print("To generate it, run the processing pipeline first:")
    print("  1) python normalize_chats.py")
    print("  2) python process_runner.py combined_ai_chat_history.csv")
    if Path('combined_ai_chat_history.csv').exists():
        print("Detected combined_ai_chat_history.csv – you can run step 2 immediately.")
    else:
        print("No combined_ai_chat_history.csv found. The normalization step will create it from your exports.")
    raise FileNotFoundError("chat_analysis.db is required. See instructions above.")

# Connect to the processed database
conn = sqlite3.connect(str(db_path))

# Verify required tables exist
required_tables = {'raw_conversations', 'message_features', 'conversation_features'}
existing_tables = set(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)['name'])
missing = required_tables - existing_tables
if missing:
    print(f"⚠️ Missing tables in database: {', '.join(sorted(missing))}")
    print("Please re-run the pipeline: python process_runner.py combined_ai_chat_history.csv")
    raise RuntimeError(f"Database is incomplete. Missing tables: {', '.join(sorted(missing))}")

# Load all data tables
print("Loading processed chat data...")

# Raw conversations
conversations_df = pd.read_sql_query("""
                                     SELECT *
                                     FROM raw_conversations
                                     """, conn)

# Message features
features_df = pd.read_sql_query("""
                                SELECT *
                                FROM message_features
                                """, conn)

# Conversation-level features
conv_features_df = pd.read_sql_query("""
                                     SELECT *
                                     FROM conversation_features
                                     """, conn)

# Combined view
combined_df = pd.read_sql_query("""
                                SELECT r.*,
                                       mf.content_type,
                                       mf.sentiment_score,
                                       mf.has_code,
                                       mf.has_urls,
                                       mf.has_questions
                                FROM raw_conversations r
                                         JOIN message_features mf ON r.message_id = mf.message_id
                                """, conn)

# Convert timestamps
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], utc=True, errors='coerce')
conversations_df['timestamp'] = pd.to_datetime(conversations_df['timestamp'], utc=True, errors='coerce')

print(f"✅ Loaded {len(conversations_df):,} messages from {len(conv_features_df):,} conversations")
print(f"📈 Data spans from {combined_df['timestamp'].min()} to {combined_df['timestamp'].max()}")

Loading processed chat data...


ValueError: time data "2025-07-16 13:18:47+00:00" doesn't match format "%Y-%m-%d %H:%M:%S.%f%z", at position 14598. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## 📊 Overview Statistics

In [ ]:
# Basic statistics
print("=== CHAT ANALYSIS OVERVIEW ===")
print(f"📝 Total Messages: {len(combined_df):,}")
print(f"💬 Total Conversations: {combined_df['conversation_id'].nunique():,}")
print(f"🤖 AI Sources: {', '.join(combined_df['source_ai'].unique())}")
print(
    f"📅 Date Range: {combined_df['timestamp'].min().strftime('%Y-%m-%d')} to {combined_df['timestamp'].max().strftime('%Y-%m-%d')}")
print(f"📊 Total Words: {combined_df['word_count'].sum():,}")
print(f"📈 Average Words per Message: {combined_df['word_count'].mean():.1f}")

print("\n=== MESSAGE BREAKDOWN ===")
role_counts = combined_df['role'].value_counts()
for role, count in role_counts.items():
    print(f"{role}: {count:,} messages ({count / len(combined_df) * 100:.1f}%)")

print("\n=== CONTENT TYPES ===")
content_counts = combined_df['content_type'].value_counts()
for content_type, count in content_counts.items():
    print(f"{content_type.title()}: {count:,} messages ({count / len(combined_df) * 100:.1f}%)")

## 📈 Activity Patterns

In [ ]:
# Messages over time
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Messages Over Time', 'Messages by AI Source', 'Content Types Distribution',
                    'Sentiment Distribution'],
    specs=[[{"secondary_y": True}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "histogram"}]]
)

# Messages over time by AI source
daily_messages = combined_df.groupby([combined_df['timestamp'].dt.date, 'source_ai']).size().reset_index(name='count')
for ai_source in combined_df['source_ai'].unique():
    ai_data = daily_messages[daily_messages['source_ai'] == ai_source]
    fig.add_trace(
        go.Scatter(x=ai_data['timestamp'], y=ai_data['count'], name=ai_source, mode='lines+markers'),
        row=1, col=1
    )

# Messages by AI source
ai_counts = combined_df['source_ai'].value_counts()
fig.add_trace(
    go.Bar(x=ai_counts.index, y=ai_counts.values, name='AI Sources'),
    row=1, col=2
)

# Content types pie chart
fig.add_trace(
    go.Pie(labels=content_counts.index, values=content_counts.values, name="Content Types"),
    row=2, col=1
)

# Sentiment distribution
fig.add_trace(
    go.Histogram(x=combined_df['sentiment_score'], name='Sentiment', nbinsx=30),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Chat Analysis Dashboard")
fig.show()

## 🕒 Temporal Analysis

In [ ]:
# Add time-based features
combined_df['hour'] = combined_df['timestamp'].dt.hour
combined_df['day_of_week'] = combined_df['timestamp'].dt.day_name()
combined_df['month'] = combined_df['timestamp'].dt.month_name()

# Hourly activity pattern
hourly_activity = combined_df.groupby(['hour', 'source_ai']).size().reset_index(name='count')

fig = px.line(hourly_activity, x='hour', y='count', color='source_ai',
              title='Chat Activity by Hour of Day',
              labels={'hour': 'Hour of Day', 'count': 'Number of Messages'})
fig.show()

# Day of week activity
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_activity = combined_df.groupby(['day_of_week', 'source_ai']).size().reset_index(name='count')
daily_activity['day_of_week'] = pd.Categorical(daily_activity['day_of_week'], categories=day_order, ordered=True)
daily_activity = daily_activity.sort_values('day_of_week')

fig = px.bar(daily_activity, x='day_of_week', y='count', color='source_ai',
             title='Chat Activity by Day of Week',
             labels={'day_of_week': 'Day of Week', 'count': 'Number of Messages'})
fig.show()

## 💬 Conversation Analysis

In [ ]:
# Conversation lengths and characteristics
print("=== CONVERSATION CHARACTERISTICS ===")
print(f"Average messages per conversation: {conv_features_df['message_count'].mean():.1f}")
print(f"Median messages per conversation: {conv_features_df['message_count'].median():.1f}")
print(f"Average words per conversation: {conv_features_df['total_word_count'].mean():.0f}")
print(f"Average conversation duration: {conv_features_df['duration_minutes'].mean():.1f} minutes")

# Conversation types
print("\n=== CONVERSATION TYPES ===")
conv_type_counts = conv_features_df['conversation_type'].value_counts()
for conv_type, count in conv_type_counts.items():
    print(
        f"{conv_type.replace('_', ' ').title()}: {count:,} conversations ({count / len(conv_features_df) * 100:.1f}%)")

# Visualize conversation characteristics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Conversation Length Distribution', 'Duration vs Messages', 'Conversation Types',
                    'Complexity vs Idea Density']
)

# Conversation length distribution
fig.add_trace(
    go.Histogram(x=conv_features_df['message_count'], name='Message Count', nbinsx=20),
    row=1, col=1
)

# Duration vs Messages scatter
fig.add_trace(
    go.Scatter(x=conv_features_df['message_count'], y=conv_features_df['duration_minutes'],
               mode='markers', name='Duration vs Messages'),
    row=1, col=2
)

# Conversation types
fig.add_trace(
    go.Bar(x=conv_type_counts.index, y=conv_type_counts.values, name='Conversation Types'),
    row=2, col=1
)

# Complexity vs. Idea Density
fig.add_trace(
    go.Scatter(x=conv_features_df['complexity_score'], y=conv_features_df['idea_density'],
               mode='markers', name='Complexity vs Density'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Conversation Analysis")
fig.show()

## 🔍 Content Analysis

In [ ]:
# Content analysis by AI source
content_by_ai = combined_df.groupby(['source_ai', 'content_type']).size().reset_index(name='count')
content_pivot = content_by_ai.pivot(index='source_ai', columns='content_type', values='count').fillna(0)

# Normalize by percentage
content_pivot_pct = content_pivot.div(content_pivot.sum(axis=1), axis=0) * 100

fig = px.bar(content_by_ai, x='source_ai', y='count', color='content_type',
             title='Content Types by AI Source',
             labels={'source_ai': 'AI Source', 'count': 'Number of Messages'})
fig.show()

# Code vs non-code analysis
code_analysis = combined_df.groupby('source_ai').agg({
    'has_code': 'sum',
    'has_urls': 'sum',
    'has_questions': 'sum',
    'message_id': 'count'
}).reset_index()

code_analysis['code_percentage'] = (code_analysis['has_code'] / code_analysis['message_id']) * 100
code_analysis['url_percentage'] = (code_analysis['has_urls'] / code_analysis['message_id']) * 100
code_analysis['question_percentage'] = (code_analysis['has_questions'] / code_analysis['message_id']) * 100

print("=== CONTENT CHARACTERISTICS BY AI SOURCE ===")
for _, row in code_analysis.iterrows():
    print(f"\n{row['source_ai']}:")
    print(f"  Code messages: {row['code_percentage']:.1f}%")
    print(f"  URL messages: {row['url_percentage']:.1f}%")
    print(f"  Question messages: {row['question_percentage']:.1f}%")

# Visualize content characteristics
fig = go.Figure()
fig.add_trace(go.Bar(name='Code', x=code_analysis['source_ai'], y=code_analysis['code_percentage']))
fig.add_trace(go.Bar(name='URLs', x=code_analysis['source_ai'], y=code_analysis['url_percentage']))
fig.add_trace(go.Bar(name='Questions', x=code_analysis['source_ai'], y=code_analysis['question_percentage']))
fig.update_layout(title='Content Characteristics by AI Source (%)', barmode='group')
fig.show()

## 😊 Sentiment Analysis

In [ ]:
# Sentiment analysis by AI and content type
sentiment_stats = combined_df.groupby(['source_ai', 'role']).agg({
    'sentiment_score': ['mean', 'std', 'count']
}).round(3)

print("=== SENTIMENT ANALYSIS ===")
print(sentiment_stats)

# Sentiment over time
combined_df['date'] = combined_df['timestamp'].dt.date
daily_sentiment = combined_df.groupby(['date', 'source_ai'])['sentiment_score'].mean().reset_index()

fig = px.line(daily_sentiment, x='date', y='sentiment_score', color='source_ai',
              title='Average Sentiment Over Time',
              labels={'date': 'Date', 'sentiment_score': 'Average Sentiment Score'})
fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Neutral")
fig.show()

# Sentiment distribution by AI source
fig = px.box(combined_df, x='source_ai', y='sentiment_score', color='source_ai',
             title='Sentiment Score Distribution by AI Source')
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.show()

# Sentiment by content type
fig = px.box(combined_df, x='content_type', y='sentiment_score',
             title='Sentiment by Content Type')
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_xaxes(tickangle=45)
fig.show()

## 🏆 Top Insights

In [ ]:
# Most productive days
daily_counts = combined_df.groupby(combined_df['timestamp'].dt.date).size()
top_days = daily_counts.nlargest(10)

print("=== TOP 10 MOST ACTIVE DAYS ===")
for date, count in top_days.items():
    print(f"{date}: {count} messages")

# Longest conversations
longest_convs = conv_features_df.nlargest(10, 'message_count')[
    ['title', 'source_ai', 'message_count', 'total_word_count', 'conversation_type']]
print("\n=== TOP 10 LONGEST CONVERSATIONS ===")
for _, conv in longest_convs.iterrows():
    title = conv['title'][:50] + "..." if len(str(conv['title'])) > 50 else conv['title']
    print(f"{title} ({conv['source_ai']})")
    print(f"  {conv['message_count']} messages, {conv['total_word_count']:,} words, type: {conv['conversation_type']}")

# Most complex conversations (high code and question content)
complex_convs = conv_features_df.nlargest(10, 'complexity_score')[
    ['title', 'source_ai', 'complexity_score', 'conversation_type']]
print("\n=== TOP 10 MOST COMPLEX CONVERSATIONS ===")
for _, conv in complex_convs.iterrows():
    title = conv['title'][:50] + "..." if len(str(conv['title'])) > 50 else conv['title']
    print(f"{title} ({conv['source_ai']})")
    print(f"  Complexity: {conv['complexity_score']:.2f}, Type: {conv['conversation_type']}")

## 🎯 Custom Analysis

Use this section to run your own custom queries and analyses.

In [ ]:
# Example: Find all conversations about a specific topic
search_term = "python"  # Change this to search for specific topics

topic_conversations = combined_df[combined_df['content'].str.contains(search_term, case=False, na=False)]
topic_conv_ids = topic_conversations['conversation_id'].unique()

print(f"Found {len(topic_conv_ids)} conversations mentioning '{search_term}'")
print(f"Total messages in these conversations: {len(topic_conversations)}")

# Show some examples
if len(topic_conv_ids) > 0:
    sample_convs = conv_features_df[conv_features_df['conversation_id'].isin(topic_conv_ids)].head(5)
    print(f"\nSample conversations:")
    for _, conv in sample_convs.iterrows():
        title = conv['title'][:60] + "..." if len(str(conv['title'])) > 60 else conv['title']
        print(f"• {title} ({conv['source_ai']}) - {conv['message_count']} messages")

In [ ]:
# Export summary data for external use
summary_data = {
    'total_messages': len(combined_df),
    'total_conversations': len(conv_features_df),
    'ai_sources': list(combined_df['source_ai'].unique()),
    'date_range': {
        'start': combined_df['timestamp'].min().isoformat(),
        'end': combined_df['timestamp'].max().isoformat()
    },
    'content_types': content_counts.to_dict(),
    'avg_sentiment': combined_df['sentiment_score'].mean(),
    'code_percentage': (combined_df['has_code'].sum() / len(combined_df)) * 100
}

print("=== EXPORTABLE SUMMARY ===")
print(json.dumps(summary_data, indent=2, default=str))

# Optionally save to a file
# with open('chat_analysis_summary.json', 'w') as f:
#     json.dump(summary_data, f, indent=2, default=str)
# print("Summary saved to chat_analysis_summary.json")

## 🔧 Database Connection Cleanup

In [ ]:
# Close database connection
conn.close()
print("✅ Database connection closed")
print("🎉 Analysis complete! Feel free to modify and extend this notebook for your specific needs.")